<a href="https://colab.research.google.com/github/savian618/Reporting/blob/main/IDP_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IDP Report Script
This script utilizes the current day's dataset to collect information about the progress of the IDP intiative.

Results are populated in the following sheet:
https://docs.google.com/spreadsheets/d/1GKWFicVSaNnI8TmS6p1b1anFruvO8MpsgpgqNgLLuCM/edit?usp=drive_link


In [1]:
!pip install --upgrade -q gspread

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 1.8 MB/s eta 0:00:00


## Mount your drive

In [2]:
from google.colab import auth, drive
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

import pandas as pd

import os
import glob
from gspread_dataframe import set_with_dataframe

gc = gspread.authorize(creds)
drive.mount('/content/drive')

Mounted at /content/drive


## Merge All Loans


In [16]:
csv_directory = 'path/to/your/csv/files'

csv_files = glob.glob(os.path.join(csv_directory, '*.csv'))

df = pd.DataFrame()

for file in csv_files:
    # Read the CSV into a DataFrame
    temp = pd.read_csv(file)

    # Append the DataFrame to the merged DataFrame
    df = df.append(temp, ignore_index=True)

In [5]:
df1 = pd.read_csv('Loan82.csv')
df2 = pd.read_csv('Loan75.csv')
df =pd.concat([df1, df2], ignore_index=True)

## Get Today's Dataset

In [3]:
#Change to today's dataset
df = pd.read_csv('Pages.csv')
df = df[['Package ID','RA Label', 'Predicated Label', 'Page Number', 'Representation']]
#define top 60 docs
df_60 = pd.read_csv('/content/drive/My Drive/top60.csv')
df_60['Priority Labels'] = df_60['Priority Labels'].str.lower()
top60 = df_60['Priority Labels'].tolist()

#Properly sort dataset
df = df.sort_values(by=['Package ID', 'Page Number'], ascending=True)

#fill the empty spots left by IDP
df['Predicated Label'].fillna(method='ffill', inplace=True)
df['Predicated Label'] = df['Predicated Label'].str.lower()
df['RA Label'].fillna('miscellaneous', inplace=True)

#Fix match column now that its filled
df['Match'] = df['RA Label'] == df['Predicated Label']

df['Top60'] = df['RA Label'].isin(top60)
df['UniqueDocFlag'] = df['RA Label'].shift() != df['RA Label']
id_counter = 1
doc_ids = []
for val in df['UniqueDocFlag']:
    if val:
        id_counter += 1
    doc_ids.append(id_counter)
df['DocumentID'] = doc_ids
df['DocPass'] = df.groupby('DocumentID')['Match'].transform(lambda x:  all(x))
df['DocFail'] = (df.groupby('DocumentID')['DocPass']
                         .transform('idxmin') == df.index) & ~df['DocPass']
df['DocFail'] = df['DocFail'].astype(int)
def first_page(row):
    if row['UniqueDocFlag'] and not row['Match']:
        return 1
    else:
        return 0
df['FirstPageFail'] = df.apply(first_page, axis=1)
"""
df_type = pd.read_csv('/content/drive/My Drive/Standard.csv')
standard_doc_types = df_type['Priority Labels'].str.lower().tolist()
def categorize_doc_type(doc_type):
    if doc_type in standard_doc_types:
        return 'standard'
    else:
        return 'non-standard'
df['Doc Type Category'] = df['RA Label'].apply(categorize_doc_type)
"""
df['Representation'].fillna('varied', inplace=True)
df =df.reset_index()
del df['index']
display(df)

,Package ID,RA Label,Predicated Label,Page Number,Representation,Match,Top60,UniqueDocFlag,DocumentID,DocPass,DocFail,FirstPageFail
0,8205592275,wire instructions,wire instructions,1,varied,True,False,True,2,True,0,0
1,8205592275,note,note,2,standard,True,True,True,3,True,0,0
2,8205592275,note,note,3,standard,True,True,False,3,True,0,0
3,8205592275,note,note,4,standard,True,True,False,3,True,0,0
4,8205592275,note,note,5,standard,True,True,False,3,True,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1115,8205935982,itemization of settlement charges,itemization of settlement charges,500,standard,True,False,True,307,True,0,0
1116,8205935982,itemization of settlement charges,itemization of settlement charges,501,standard,True,False,False,307,True,0,0
1117,8205935982,itemization of settlement charges,itemization of settlement charges,502,standard,True,False,False,307,True,0,0
1118,8205935982,amendatory clause,amendatory clause,503,varied,True,False,True,308,True,0,0


## Update all_Pages_dataset_133


In [4]:
workbook = gc.open('IDP Report')
worksheet = workbook.worksheet("Pages")
worksheet.clear()
set_with_dataframe(worksheet, df)

##Add Pivot table to daily results

In [5]:
workbook = gc.open('IDP Report')
worksheet = workbook.worksheet("Today's Results")
df = pd.DataFrame(worksheet.get_all_records(head=1))
#Change to the date of the dataset
df['Date'] = '5/1/2024'
display(df)

,RA Label,Top 60,Standard vs Non-Standard,Total Pages,Page Mismatches,Pg Accuracy,Unique Doc Ct,Doc Failed,Doc Accuracy,Total First Pages,First Page Mismatches,First Page Accuracy,Loans,Date
0,11 and 22 seem to be continuation of the same ...,FALSE,varied,1,1,0.00%,1,1,0.00%,1,1,0.00%,1,5/1/2024
1,addendum to closing disclosure,FALSE,standard,5,0,100.00%,5,0,100.00%,5,0,100.00%,2,5/1/2024
2,addendum to loan estimate,FALSE,standard,1,1,0.00%,1,1,0.00%,1,1,0.00%,1,5/1/2024
3,address certification,FALSE,varied,4,0,100.00%,3,0,100.00%,3,0,100.00%,2,5/1/2024
4,affidavit other,FALSE,varied,3,0,100.00%,3,0,100.00%,3,0,100.00%,2,5/1/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,w-9,FALSE,standard,12,0,100.00%,2,0,100.00%,2,0,100.00%,2,5/1/2024
147,warranty of completion of construction,FALSE,standard,5,0,100.00%,2,0,100.00%,2,0,100.00%,1,5/1/2024
148,wire instructions,FALSE,varied,1,0,100.00%,1,0,100.00%,1,0,100.00%,1,5/1/2024
149,your home loan toolkit,FALSE,varied,29,0,100.00%,2,0,100.00%,2,0,100.00%,2,5/1/2024


In [6]:
workbook = gc.open('IDP Report')
worksheet = workbook.worksheet("Daily Results")
existing_data = worksheet.get_all_records()
if existing_data:
    start_row = len(existing_data) + 2
else:
    start_row = 1
set_with_dataframe(worksheet, df, row=start_row, include_index=False, include_column_header=False)

## Dashboard Update

Get the Daily comparison numbers for Dashboard updates

In [14]:
def calculate_diff(group):
    try:
      group['Doc Accuracy Diff'] = group['Doc Accuracy'] - group['Doc Accuracy'].shift()
    except:
      group['Doc Accuracy Diff'] = 0
    return group

In [7]:
workbook = gc.open("IDP Report")
worksheet = workbook.worksheet("Daily Results")
df = pd.DataFrame(worksheet.get_all_records(head=1))

df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df = df.sort_values(by='Date', ascending=False)
rDates = df['Date'].unique().tolist()
df['Recent'] = df['Date'].isin(rDates[:8])
df = df.sort_values(by='Date', ascending=True)
"""
df['Doc Accuracy'] = df['Doc Accuracy'].str.replace('%', '')
df['Doc Accuracy'] = df['Doc Accuracy'].astype(float)
df['Doc Accuracy'] = df['Doc Accuracy'] / 100
df = df.groupby(['RA Label']).apply(calculate_diff)
"""
display(df)
set_with_dataframe(worksheet, df)

,RA Label,Top 60,Standard vs Non-Standard,Total Pages,Page Mismatches,Pg Accuracy,Unique Doc Ct,Doc Failed,Doc Accuracy,Total First Pages,First Page Mismatches,First Page Accuracy,Loans,Date,Recent
0,acknowledgement of no inspection,FALSE,varied,1,1,0.00%,1,1,0.00%,1,1,0.00%,1,2024-04-30,True
29,credit report,TRUE,varied,9,1,88.89%,2,1,50.00%,2,0,100.00%,1,2024-04-30,True
2,allonge to note,FALSE,varied,3,0,100.00%,2,0,100.00%,2,0,100.00%,1,2024-04-30,True
3,alta settlement statement,FALSE,standard,3,0,100.00%,1,0,100.00%,1,0,100.00%,1,2024-04-30,True
4,amendatory clause,FALSE,varied,1,0,100.00%,1,0,100.00%,1,0,100.00%,1,2024-04-30,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,w-9,FALSE,standard,12,0,100.00%,2,0,100.00%,2,0,100.00%,2,2024-05-01,True
268,warranty of completion of construction,FALSE,standard,5,0,100.00%,2,0,100.00%,2,0,100.00%,1,2024-05-01,True
269,wire instructions,FALSE,varied,1,0,100.00%,1,0,100.00%,1,0,100.00%,1,2024-05-01,True
256,urla - unmarried addendum,FALSE,standard,4,0,100.00%,4,0,100.00%,4,0,100.00%,1,2024-05-01,True
